In [2]:
import dill
dill.load_session("metodaysmall.db")

C:\Users\nroy0\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
def computeMixedAccuracyLog2(posPatternClass, unigramsClass,  mixedClass, finalFeatures, finalFeaturesMSE, cross_val,c, cls):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    originalclass = []
    predictedclass = []
    
    yclass = finalFeatures.iloc[:,-1]
    posPatternMMSE = normalize(posPatternClass)
    unigramsMMSE = normalize(unigramsClass)
    mixedMMSE = normalize(mixedClass)
    finalFeaturesNormMSE = normalize(finalFeatures)
    
    #only works with nd numpy arrays
    
    ycont = finalFeaturesMSE.truthMean.values
    yp = posPatternClass.truthClass.values
    yu = unigramsClass.truthClass.values
    ym = mixedClass.truthClass.values
    yf = finalFeatures.truthClass.values
    
    Xp = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns)
    Xu = unigramsMMSE.as_matrix(columns = unigramsMMSE.columns)
    Xm = mixedMMSE.as_matrix(columns = mixedMMSE.columns)
    Xf = finalFeaturesNormMSE.as_matrix(columns = finalFeaturesNormMSE.columns)
    

    Xp_new = SelectKBest(f_classif, k="all").fit_transform(Xp, yp)
    Xu_new = SelectKBest(f_classif, k=250).fit_transform(Xu, yu)
    Xm_new = SelectKBest(f_classif, k="all").fit_transform(Xm, ym)
    Xf_new = SelectKBest(f_classif, k="all").fit_transform(Xf, yf)
    
    
    y_out = pd.DataFrame()
#     scores = []
    accM = []
    accS = []
#     scoresM = []
    count = 0
    
    df = pd.DataFrame(np.random.randn(2459, 2))

    msk = np.random.rand(len(df)) < 0.8


    clrp = SVC(probability = True)
    clru = LogisticRegressionCV()
    clrm = SVC(probability = True)
    clrf = LogisticRegressionCV()

    clrp.fit(Xp_new[msk], yp[msk])
    clru.fit(Xu_new[msk], yu[msk])
    clrm.fit(Xm_new[msk], ym[msk])
    clrf.fit(Xf_new[msk], yf[msk])

    yp_pred = clrp.predict_proba(Xp_new[~msk])[:,1]
    yp_pred_train = clrp.predict_proba(Xp_new[msk])[:,1]
#             yp_pred = clrp.predict(Xp_new[val_idx])
#             yp_pred_train = clrp.predict(Xp_new[train_idx])

#             yu_pred = clru.predict(Xu_new[val_idx])
#             yu_pred_train = clru.predict(Xu_new[train_idx])

#             ym_pred = clrm.predict(Xm_new[val_idx])
#             ym_pred_train = clrm.predict(Xm_new[train_idx])

#             yf_pred = clrf.predict(Xf_new[val_idx])
#             yf_pred_train = clrf.predict(Xf_new[train_idx])


    yu_pred = clru.predict_proba(Xu_new[~msk])[:,1]
    yu_pred_train = clru.predict_proba(Xu_new[msk])[:,1]

    ym_pred = clrm.predict_proba(Xm_new[~msk])[:,1]
    ym_pred_train = clrm.predict_proba(Xm_new[msk])[:,1]


    yf_pred = clrf.predict_proba(Xf_new[~msk])[:,1]
    yf_pred_train = clrf.predict_proba(Xf_new[msk])[:,1]

#             y_out["c"+str(count)]=y_pred
#             count = count + 1

    d1= {'p':yp_pred.tolist(), 'u':yu_pred.tolist(), 'm':ym_pred.tolist(), 'f':yf_pred.tolist(), 'y': yclass[~msk].tolist(), 'ycont': ycont[~msk].tolist()}
    d2= {'p':yp_pred_train.tolist(), 'u':yu_pred_train.tolist(), 'm':ym_pred_train.tolist(), 'f':yf_pred_train.tolist(), 'y': yclass[msk].tolist(), 'ycont': ycont[msk].tolist()}
    df1 = pd.DataFrame(d1)
    df2 = pd.DataFrame(d2)

    d= pd.concat([df1, df2])

    X_train, y_train, ycont_train = df2[["p", "u", "m", "f"]], df2["y"], df2["ycont"]
    #test = d[~msk]
    X_test, y_test, ycont_test = df1[["p", "u", "m", "f"]], df1["y"], df1["ycont"]

    if(cls == "RF"):
        clf = RandomForestRegressor(max_depth=c, random_state=0) #i used random forest
        #             prob = cl.predict_proba
        clf.fit(X_train, ycont_train) # i changed this to work with regression
        #score = clf.score(X_test, ycont_test) ## i removed this so that it worked with regression
        prob = clf.predict(X_test)


    elif(cls == "SVC"):
        clf =  LinearSVR(C=c, loss='squared_epsilon_insensitive', dual=False, random_state=1)
        #             prob = cl.predict_proba
        clf.fit(X_train, ycont_train) # i changed this to work with regression
        #score = clf.score(X_test, ycont_test) ## i removed this so that it worked with regression
        prob = clf.predict(X_test)


    elif(cls == "Log"):
        clf = LogisticRegressionCV()
        clf.fit(X_train, y_train)
        prob = clf.predict_proba(X_test)[:,1]
    elif(cls == "Xg"):
        clf = GradientBoostingClassifier()
        clf.fit(X_train, y_train)
        prob = clf.predict_proba(X_test)[:,1] 
    elif(cls == "XgBoost"):
        clf =  xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                   colsample_bytree=1, max_depth=7)
        #             prob = cl.predict_proba
        clf.fit(X_train, ycont_train) # i changed this to work with regression
        #score = clf.score(X_test, ycont_test) ## i removed this so that it worked with regression
        prob = clf.predict(X_test)


    lab_pred = prob.copy() # i added this to work with regression
    lab_pred=np.where(lab_pred >= 0.5, 1,-1)
    score = accuracy_score(lab_pred,y_test)
    rmse = mean_squared_error(ycont_test, prob)
#             accuracyM = scores.mean()
#             accuracyS = scores.std()
    accM.append(score)
    accS.append(rmse)

    originalclass.extend(y_test)
    predictedclass.extend(lab_pred)
#             y1 = { 'y': yclass[val_idx].tolist()}
#             y2 = { 'y': yclass[train_idx].tolist()}
#             yf1 = pd.DataFrame(y1)
#             yf2 = pd.DataFrame(y2)
#             ylab = pd.concat([y1, y2]) 

#             y_pred_final = pd.DataFrame(d)
#             y_pred_final = pd.DataFrame(d)
#             y_pred_final["mean"] = y_pred_final.mean(axis=1)
#             y_pred_final["median"] = y_pred_final.iloc[:,0:4].median(axis=1)
#             y_pred_final['class'] = np.where(y_pred_final['mean'] >= 0.5, 1,-1)
# #             y_pred_final['class'] = np.where(y_pred_final['median'] >= 0.5, 1,-1)

#             rmse = mean_squared_error(ycont[val_idx], y_pred_final["mean"])
#             scores.append(rmse)

#             rmsemed = mean_squared_error(ycont[val_idx], y_pred_final["median"])
#             scoresM.append(rmsemed)

#             accuracy = accuracy_score(yclass[val_idx], y_pred_final["class"])
#             acc.append(accuracy)

#             rmse = mean_squared_error(y[val_idx], y_pred)
#             scores.append(rmse)

    mS = np.mean(accS)
    sS = np.std(accS)
    print("MSE: %0.3f (+/- %0.2f)" % (mS, sS))

#     return(d)
    mA = np.mean(accM)
    sA = np.std(accM)
    print("Accuracy: %0.3f (+/- %0.2f)" % (mA, sA))
    
    print(classification_report(originalclass, predictedclass)) 
    
#     mMed = np.mean(scoresM)
#     sMed = np.std(scoresM)
#     print("MSEmedian: %0.3f (+/- %0.2f)" % (mMed, sMed))

In [33]:
    originalclass = []
    predictedclass = []
    
    yclass = finalFeatures.iloc[:,-1]
    posPatternMMSE = normalize(posPatternClass)
    unigramsMMSE = normalize(unigramsClass)
    mixedMMSE = normalize(mixedClass)
    finalFeaturesNormMSE = normalize(finalFeatures)
    
    #only works with nd numpy arrays
    
    ycont = finalFeaturesMSE.truthMean.values
    yp = posPatternClass.truthClass.values
    yu = unigramsClass.truthClass.values
    ym = mixedClass.truthClass.values
    yf = finalFeatures.truthClass.values
    
    Xp = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns)
    Xu = unigramsMMSE.as_matrix(columns = unigramsMMSE.columns)
    Xm = mixedMMSE.as_matrix(columns = mixedMMSE.columns)
    Xf = finalFeaturesNormMSE.as_matrix(columns = finalFeaturesNormMSE.columns)
    

    Xp_new = SelectKBest(f_classif, k="all").fit_transform(Xp, yp)
    Xu_new = SelectKBest(f_classif, k=250).fit_transform(Xu, yu)
    Xm_new = SelectKBest(f_classif, k="all").fit_transform(Xm, ym)
    Xf_new = SelectKBest(f_classif, k="all").fit_transform(Xf, yf)
    
    
    y_out = pd.DataFrame()
#     scores = []
    accM = []
    accS = []
#     scoresM = []
    count = 0
    
    df = pd.DataFrame(np.random.randn(2459, 2))

    msk = np.random.rand(len(df)) < 0.9


    clrp = SVC(probability = True)
    clru = LogisticRegressionCV()
    clrm = SVC(probability = True)
    clrf = LogisticRegressionCV()

    clrp.fit(Xp_new[msk], yp[msk])
    clru.fit(Xu_new[msk], yu[msk])
    clrm.fit(Xm_new[msk], ym[msk])
    clrf.fit(Xf_new[msk], yf[msk])

    yp_pred = clrp.predict_proba(Xp_new[~msk])[:,1]
    yp_pred_train = clrp.predict_proba(Xp_new[msk])[:,1]
#             yp_pred = clrp.predict(Xp_new[val_idx])
#             yp_pred_train = clrp.predict(Xp_new[train_idx])

#             yu_pred = clru.predict(Xu_new[val_idx])
#             yu_pred_train = clru.predict(Xu_new[train_idx])

#             ym_pred = clrm.predict(Xm_new[val_idx])
#             ym_pred_train = clrm.predict(Xm_new[train_idx])

#             yf_pred = clrf.predict(Xf_new[val_idx])
#             yf_pred_train = clrf.predict(Xf_new[train_idx])


    yu_pred = clru.predict_proba(Xu_new[~msk])[:,1]
    yu_pred_train = clru.predict_proba(Xu_new[msk])[:,1]

    ym_pred = clrm.predict_proba(Xm_new[~msk])[:,1]
    ym_pred_train = clrm.predict_proba(Xm_new[msk])[:,1]


    yf_pred = clrf.predict_proba(Xf_new[~msk])[:,1]
    yf_pred_train = clrf.predict_proba(Xf_new[msk])[:,1]

#             y_out["c"+str(count)]=y_pred
#             count = count + 1

    d1= {'p':yp_pred.tolist(), 'u':yu_pred.tolist(), 'm':ym_pred.tolist(), 'f':yf_pred.tolist(), 'y': yclass[~msk].tolist(), 'ycont': ycont[~msk].tolist()}
    d2= {'p':yp_pred_train.tolist(), 'u':yu_pred_train.tolist(), 'm':ym_pred_train.tolist(), 'f':yf_pred_train.tolist(), 'y': yclass[msk].tolist(), 'ycont': ycont[msk].tolist()}
    df1 = pd.DataFrame(d1)
    df2 = pd.DataFrame(d2)

    d= pd.concat([df1, df2])

    X_train, y_train, ycont_train = df2[["p", "u", "m", "f"]], df2["y"], df2["ycont"]
    #test = d[~msk]
    X_test, y_test, ycont_test = df1[["p", "u", "m", "f"]], df1["y"], df1["ycont"]


In [34]:
clf =  LinearSVR(C=c, loss='squared_epsilon_insensitive', dual=False, random_state=1)
        #             prob = cl.predict_proba
clf.fit(X_train, ycont_train) # i changed this to work with regression
#score = clf.score(X_test, ycont_test) ## i removed this so that it worked with regression
prob = clf.predict(X_test)
lab_pred = prob.copy() # i added this to work with regression
lab_pred=np.where(lab_pred >= 0.5, 1,-1)
score = accuracy_score(lab_pred,y_test)
score

0.74

In [35]:
misclassified = np.where(y_test != lab_pred)
misclassified

(array([  2,   7,   9,  17,  21,  26,  27,  28,  29,  30,  31,  38,  48,
         53,  55,  56,  59,  65,  67,  71,  76,  85,  87,  88, 104, 108,
        112, 127, 128, 130, 139, 140, 141, 142, 157, 158, 160, 162, 168,
        183, 185, 186, 187, 194, 196, 200, 201, 206, 207, 214, 218, 219,
        220, 222, 225, 226, 227, 228, 232, 233, 240, 241, 242, 244, 249],
       dtype=int64),)

In [37]:
mis = varpd[~msk]
pp= mis.iloc[misclassified]
pp = pd.merge(pp, Labels[["id","truthClass", "truthMean"]], on = "id")
pp.to_csv("bar_miscl.csv")